In [ ]:
# 주피터 노트북 구버전
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:97% !important;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.output {font-size:22pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:22pt;}
div.prompt {min-width:70px;}
</style>
"""))

In [1]:
# 주피터 노트북 새버전
from IPython.display import display, HTML
display(HTML("""
<style>
div.jp-WindowedPanel-outer.lm-Widget{padding:5px;}
div.lm-Widget{font-size:15pt;}
div.cm-content {font-family:Consolas; font-size:19pt;}
div.jp-OutputArea-output pre {font-size:19pt; font-weight:bold;}
input.jp-Stdin-input {font-family:Consolas; font-size:19pt;}
td,th{font-size:15pt;}
</style>
"""))

<b><font size="6" color="red">ch14. 웹데이터 수집</font></b>

# 1절. BeautifulSoup과 parser
`pip install bs4` 아나콘다 설치하면 자동 설치되는 패키지 7500개에 포함

`pip install requests_file`

공식 사이트 : https://www.crummy.com/software/BeautifulSoup <br>
Documentation : https://www.crummy.com/software/BeautifulSoup/bs4/doc

In [ ]:
import requests
from requests_file import FileAdapter

In [ ]:
s = requests.Session()
s.mount("file://", FileAdapter())
response = s.get('file:///D:/C_Python_RAPA/source/python/ch12_sample.html')
# D:/C_Python_RAPA/source/python/ch12_sample.html
response

In [ ]:
if response.status_code == 200:
    print('success!')
if response.status_code == 404:
    print('Not found')

In [ ]:
response.content.decode('utf-8')

In [ ]:
# html 파싱
from bs4 import BeautifulSoup
soup=BeautifulSoup(response.content,"html.parser")
soup

In [ ]:
el = soup.select_one('h1')
print(el)
print('el.text :', el.text)
print('el의 속성들 :', el.attrs)
print('el의 class속성 :', el.attrs['class'])
print('el의 name :', el.name)

In [ ]:
# h1태그들 다 : 없는 태그는 빈스트링. 한 엘리먼트만 있어도 리스트로 반환
el = soup.select("h1")
print('el : ', el)
print('el의 text들 :', [e.text for e in el])
print('el의 string :', [e.string for e in el])
print('el의 속성들 :', [e.attrs for e in el])
print('el의 class속성들 :', [e.attrs['class'] for e in el])

# 2절. 정적 웹 데이터 수집
## 2.1 request 모듈 : 네이버 환율정보 가져오기

In [ ]:
import requests
from bs4 import BeautifulSoup
from numpy import round
url = "https://finance.naver.com/marketindex"
marketindex = requests.get(url)
soup = BeautifulSoup(marketindex.content, "html.parser")
price = soup.select("div.head_info > span.value")
for idx in range(len(price)):
    print(price[idx].text)

In [ ]:
title = soup.select('h3.h_lst > span.blind')
for idx in range(len(title)):
    print(title[idx].text)

In [ ]:
unit = soup.select('div.head_info > span > span.blind')
unit = [u.text for u in unit]
unit.insert(7,'')
unit

In [ ]:
status = soup.select('div.head_info > span.blind')
for idx in range(len(status)):
    print(status[idx].text, end=', ')

In [ ]:
len(title), len(price), len(unit), len(status)

In [ ]:
for idx in range(len(title)):
    print("{} : {}{} - {}".format(title[idx].text, price[idx].text, 
                                 unit[idx], status[idx].text))

# 2.2 yest24의 베스트 셀러를 출력

In [ ]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
url = "http://www.yes24.com/24/category/bestseller"
# bestseller = requests.get(url)
# soup = BeautifulSoup(bestseller.content, "html.parser")
bestseller = urlopen(url)
soup = BeautifulSoup(bestseller, "html.parser")
authors_ = soup.select("div.info_row > span.info_auth.authPub")
for idx in range(len(el)):
    print(el[idx].select_one('a').text, end="\n\n")

In [ ]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
url = "http://www.yes24.com/24/category/bestseller"
# bestseller = requests.get(url)
# soup = BeautifulSoup(bestseller.content, "html.parser")
bestseller = urlopen(url)
soup = BeautifulSoup(bestseller, "html.parser")
titles=soup.select("div.item_info > div.info_row > a.gd_name")
# authors = soup.select("div.info_row > span.info_auth.authPub > a")
authors_wrap = soup.select("div.info_row > span.info_auth.authPub")
prices = soup.select("div.info_row > strong.txt_num > em.yes_b")
# 화면출력
print("순위,책이름,저자및출판사,가격")
title = []
author = []
price = []
for idx in range(len(titles)):
    authors = authors_wrap[idx].select_one('a')
    print("{},{},{},{}".format(idx+1, titles[idx].text, authors.text,  prices[idx].text))
    # print("{},{},{},{}".format(idx+1, titles[idx].text, authors[idx].text,  prices[idx].text))
    title.append(titles[idx].text)
    author.append(authors.text)
    price.append(prices[idx].text)

In [ ]:
import pandas as pd
df = pd.DataFrame({'순위':range(1,25),
                   '책이름':title,
                   '저자및출판사':author,
                   '가격':price})
df

In [ ]:
df.to_csv('data/yes24.csv', encoding='utf-8', index=False)